# UniProt Analysis

In [1]:
import $file.^.sparkinit, sparkinit._
import $file.^.pathinit, pathinit._
import $ivy.`com.github.pathikrit::better-files:3.8.0`
import $ivy.`com.databricks::spark-xml:0.7.0`
import ss.implicits._
import org.apache.spark.sql.functions._
import better.files._

Loading spark-stubs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
19/12/27 19:37:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


import $file.$          , sparkinit._

import $file.$         , pathinit._

import $ivy.$                                         

import $ivy.$                                

import ss.implicits._

import org.apache.spark.sql.functions._

## Partition

Break the raw xml up into parquet partitions (and save the schema):

In [14]:
// Download from https://www.uniprot.org/downloads (Swiss-Prot only for now)
val path = (File(DATA_CACHE_DIR) / "uniprot" / "uniprot_sprot.xml").toString
val df = ss.read
    .format("com.databricks.spark.xml")
    .option("rowTag", "entry")
    .load(path)

fold at InferSchema.scala:115

202 / 202

19/12/27 19:11:21 WARN Utils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.


path: String = "/home/eczech/data/ot/cache/uniprot/uniprot_sprot.xml"
df: org.apache.spark.sql.package.DataFrame = [_created: string, _dataset: string ... 18 more fields]

In [ ]:
df.printSchema

root
 |-- _created: string (nullable = true)
 |-- _dataset: string (nullable = true)
 |-- _modified: string (nullable = true)
 |-- _version: long (nullable = true)
 |-- _xmlns: string (nullable = true)
 |-- accession: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- comment: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- _VALUE: string (nullable = true)
 |    |    |-- _error: double (nullable = true)
 |    |    |-- _evidence: string (nullable = true)
 |    |    |-- _locationType: string (nullable = true)
 |    |    |-- _mass: double (nullable = true)
 |    |    |-- _method: string (nullable = true)
 |    |    |-- _name: string (nullable = true)
 |    |    |-- _type: string (nullable = true)
 |    |    |-- absorption: struct (nullable = true)
 |    |    |    |-- max: struct (nullable = true)
 |    |    |    |    |-- _VALUE: string (nullable = true)
 |    |    |    |    |-- _evidence: string (nullable = true)
 |    |    

In [20]:
val path = (File(DATA_CACHE_DIR) / "uniprot" / "uniprot_sprot.schema.txt")
path.overwrite(df.schema.treeString)

path: File = /home/eczech/data/ot/cache/uniprot/uniprot_sprot.schema.txt
res19_1: File = /home/eczech/data/ot/cache/uniprot/uniprot_sprot.schema.txt

In [16]:
val path = (File(DATA_CACHE_DIR) / "uniprot" / "uniprot_sprot.parquet").toString
df.write.format("parquet").save(path)

save at cmd15.sc:2

202 / 202

path: String = "/home/eczech/data/ot/cache/uniprot/uniprot_sprot.parquet"

## Load 

In [3]:
val df = ss.read.parquet((File(DATA_CACHE_DIR) / "uniprot" / "uniprot_sprot.parquet").toString)

parquet at cmd2.sc:1

1 / 1

19/12/27 19:38:47 WARN Utils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.


df: org.apache.spark.sql.package.DataFrame = [_created: string, _dataset: string ... 18 more fields]

### Organisms

In [27]:
df.select("organism").printSchema

root
 |-- organism: struct (nullable = true)
 |    |-- _evidence: string (nullable = true)
 |    |-- dbReference: struct (nullable = true)
 |    |    |-- _VALUE: string (nullable = true)
 |    |    |-- _id: long (nullable = true)
 |    |    |-- _type: string (nullable = true)
 |    |-- lineage: struct (nullable = true)
 |    |    |-- taxon: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |-- name: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- _VALUE: string (nullable = true)
 |    |    |    |-- _type: string (nullable = true)



In [35]:
df
    .withColumn("organism", explode($"organism.name"))
    .select("organism.*")
    /* Organism type frequencies:
    +----------+------+
    |     _type| count|
    +----------+------+
    |scientific|561568|
    |    common|243738|
    |   synonym| 45458|
    +----------+------+
    */
    .filter($"_type" === "scientific")
    .groupBy("_VALUE")
    .count
    .sort($"count".desc)
    .show(false)

show at cmd34.sc:8

16 / 16

show at cmd34.sc:8

1 / 1

+------------------------------------------------------+-----+
|_VALUE                                                |count|
+------------------------------------------------------+-----+
|Homo sapiens                                          |20367|
|Mus musculus                                          |17027|
|Arabidopsis thaliana                                  |15922|
|Rattus norvegicus                                     |8085 |
|Saccharomyces cerevisiae (strain ATCC 204508 / S288c) |6721 |
|Bos taurus                                            |6008 |
|Schizosaccharomyces pombe (strain 972 / ATCC 24843)   |5140 |
|Escherichia coli (strain K12)                         |4518 |
|Bacillus subtilis (strain 168)                        |4188 |
|Dictyostelium discoideum                              |4149 |
|Caenorhabditis elegans                                |4105 |
|Oryza sativa subsp. japonica                          |4070 |
|Drosophila melanogaster                               

In [7]:
val dfh = df.filter(array_contains($"organism.name", struct(
    lit("Homo sapiens").as("_VALUE"),
    lit("scientific").as("_type")
)))
dfh.count

count at cmd6.sc:5

16 / 16

count at cmd6.sc:5

1 / 1

dfh: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [_created: string, _dataset: string ... 18 more fields]
res6_1: Long = 20367L

### Protein Existence

While most of the UP entries are proteins observed at some point in a laboratory setting, a significant portion of them are also assumed to exist based on transcript measurement or some other less direct method (see https://web.expasy.org/docs/userman.html#PE_line).

In [9]:
dfh.groupBy("proteinExistence._type").count.show(false)

show at cmd8.sc:1

16 / 16

show at cmd8.sc:1

1 / 1

+----------------------------+-----+
|_type                       |count|
+----------------------------+-----+
|evidence at protein level   |15428|
|inferred from homology      |814  |
|evidence at transcript level|3373 |
|uncertain                   |592  |
|predicted                   |160  |
+----------------------------+-----+



### Evidence Codes

Evidence codes are associated with a variety of the different types of evidence provided (see https://web.expasy.org/docs/userman.html#ev_description) and they are also stated at the root level of the entries as a generalization of what curation process an entry underwent.  The codes at this level are:

- ECO:0000269 - [experimental evidence used in manual assertion](https://www.ebi.ac.uk/QuickGO/term/ECO:0000269)
- ECO:0000244 - [combinatorial evidence used in manual assertion](https://www.ebi.ac.uk/QuickGO/term/ECO:0000244)
- ECO:0000255 - [match to sequence model evidence used in manual assertion](https://www.ebi.ac.uk/QuickGO/term/ECO:0000255)
- ECO:0000303 - [author statement without traceable support used in manual assertion](https://www.ebi.ac.uk/QuickGO/term/ECO:0000303)
- ECO:0000305 - [curator inference used in manual assertion](https://www.ebi.ac.uk/QuickGO/term/ECO:0000305)
- ECO:0000250 - [sequence similarity evidence used in manual assertion](https://www.ebi.ac.uk/QuickGO/term/ECO:0000250)
- ECO:0000312 - [imported information used in manual assertion](https://www.ebi.ac.uk/QuickGO/term/ECO:0000312)

In [51]:
// Code frequency, independent of combinations
dfh.withColumn("evidence_codes", explode($"evidence"))
    .groupBy($"evidence_codes._type".as("code")).count
    .sort($"count".desc).show

show at cmd50.sc:3

16 / 16

show at cmd50.sc:3

1 / 1

+-----------+------+
|       code| count|
+-----------+------+
|ECO:0000269|101169|
|ECO:0000244| 43423|
|ECO:0000255| 24583|
|ECO:0000303| 23939|
|ECO:0000305| 22751|
|ECO:0000250| 18315|
|ECO:0000312|  4279|
+-----------+------+



In [54]:
// Most common combinations associated with an individual entry
dfh.select($"accession".getItem(0).as("id"), explode($"evidence").as("evidence"))
    .groupBy("id").agg(collect_set($"evidence._type").as("codes"))
    .groupBy("codes").count
    .sort($"count".desc)
    .show(10, false)

show at cmd53.sc:5

16 / 16

show at cmd53.sc:5

1 / 1

show at cmd53.sc:5

1 / 1

+-------------------------------------------------------------------------------------------+-----+
|codes                                                                                      |count|
+-------------------------------------------------------------------------------------------+-----+
|[ECO:0000305, ECO:0000255, ECO:0000303, ECO:0000244, ECO:0000250, ECO:0000269]             |2741 |
|[ECO:0000305, ECO:0000255, ECO:0000303, ECO:0000250, ECO:0000269]                          |1376 |
|[ECO:0000305, ECO:0000255, ECO:0000244, ECO:0000250, ECO:0000269]                          |1246 |
|[ECO:0000305, ECO:0000255, ECO:0000250, ECO:0000269]                                       |1116 |
|[ECO:0000305, ECO:0000255, ECO:0000303, ECO:0000244, ECO:0000269]                          |1029 |
|[ECO:0000305, ECO:0000303, ECO:0000244, ECO:0000250, ECO:0000269]                          |839  |
|[ECO:0000305, ECO:0000255, ECO:0000303, ECO:0000269]                                       |837  |


### CD Antigen Names

In [23]:
dfh.groupBy("protein.cdAntigenName._VALUE")
    .count.sort($"count".desc).show(8, false)

show at cmd22.sc:2

16 / 16

show at cmd22.sc:2

1 / 1

+------+-----+
|_VALUE|count|
+------+-----+
|null  |19969|
|CD36  |3    |
|CD32  |3    |
|CD99  |2    |
|CD87  |1    |
|CD109 |1    |
|CD98  |1    |
|CD37  |1    |
+------+-----+
only showing top 8 rows



## Comments

In [12]:
dfh.withColumn("comments", explode($"comment")).select("comments.*")
    .printSchema

root
 |-- _VALUE: string (nullable = true)
 |-- _error: double (nullable = true)
 |-- _evidence: string (nullable = true)
 |-- _locationType: string (nullable = true)
 |-- _mass: double (nullable = true)
 |-- _method: string (nullable = true)
 |-- _name: string (nullable = true)
 |-- _type: string (nullable = true)
 |-- absorption: struct (nullable = true)
 |    |-- max: struct (nullable = true)
 |    |    |-- _VALUE: string (nullable = true)
 |    |    |-- _evidence: string (nullable = true)
 |    |-- text: struct (nullable = true)
 |    |    |-- _VALUE: string (nullable = true)
 |    |    |-- _evidence: string (nullable = true)
 |-- cofactor: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- _evidence: string (nullable = true)
 |    |    |-- dbReference: struct (nullable = true)
 |    |    |    |-- _VALUE: string (nullable = true)
 |    |    |    |-- _id: string (nullable = true)
 |    |    |    |-- _type: string (nullable = true)
 |    |    |-- n

In [20]:
dfh.withColumn("comments", explode($"comment"))
    .select("comments._evidence").show(50, false)
    //.count.sort($"count".desc).show(false)

show at cmd19.sc:2

1 / 1

+---------+
|_evidence|
+---------+
|null     |
|null     |
|null     |
|null     |
|null     |
|null     |
|null     |
|null     |
|null     |
|null     |
|null     |
|null     |
|null     |
|null     |
|null     |
|null     |
|null     |
|null     |
|null     |
|null     |
|7        |
|null     |
|null     |
|null     |
|null     |
|null     |
|6        |
|null     |
|null     |
|null     |
|null     |
|null     |
|null     |
|null     |
|null     |
|null     |
|null     |
|null     |
|null     |
|null     |
|null     |
|null     |
|9        |
|null     |
|null     |
|null     |
|null     |
|null     |
|null     |
|null     |
+---------+
only showing top 50 rows



In [18]:
dfh.withColumn("comments", explode($"comment"))
    .groupBy("comments._type")
    .count.sort($"count".desc)
    .show(false)

show at cmd17.sc:4

16 / 16

show at cmd17.sc:4

1 / 1

+-----------------------------+-----+
|_type                        |count|
+-----------------------------+-----+
|interaction                  |55521|
|subcellular location         |17911|
|function                     |17451|
|similarity                   |14346|
|subunit                      |12499|
|sequence caution             |11883|
|alternative products         |10628|
|tissue specificity           |9826 |
|PTM                          |7273 |
|disease                      |6740 |
|catalytic activity           |6545 |
|domain                       |4376 |
|online information           |4165 |
|miscellaneous                |4024 |
|caution                      |2198 |
|induction                    |1885 |
|cofactor                     |1712 |
|pathway                      |1344 |
|activity regulation          |1282 |
|biophysicochemical properties|859  |
+-----------------------------+-----+
only showing top 20 rows

